### Next steps:


TODO:
1. Improve dataset classes to load batches faster to remove the CPU bottleneck in training
1. Slowly increase the number of time chunks included in training each epoch (instead of all mixed in at once)
1. Develop method for measuring wave propogation speed in image coordinates,  then find the flattened homography that makes this as constant as possible.
1. Train on much wider images now that wavespeed is more uniform
1. Create a pipeline that generates animations for each stage of the training process (normalization, clipping, learning), and compiles into a demo video.
1. Get tensorboard metrics logging working with this logger class: https://github.com/PyTorchLightning/pytorch-lightning/issues/1228#issuecomment-622963564


#### First get the basic Tensorflow Lightning training loop working

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import pytorch_lightning as pl
from surfbreak.waveform_models import LitSirenNet

tb_logger = pl.loggers.TensorBoardLogger('logs/', name="wavefunc")

trainer = pl.Trainer(
    logger=tb_logger,
    limit_val_batches=1,
    max_epochs=5,
    gpus=1 if torch.cuda.is_available() else None,
    )

wavefunc_model = LitSirenNet(hidden_features=256, hidden_layers=3, first_omega_0=1.5, hidden_omega_0=11.7, squared_slowness=0.23,
                             steps_per_vid_chunk=100, learning_rate=1e-4, grad_loss_scale=0, wave_loss_scale=4.4e-8)

trainer.fit(wavefunc_model)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type  | Params | In sizes     | Out sizes                   
---------------------------------------------------------------------------------------
0 | model          | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
1 | slowness_model | Siren | 8 K    | ?            | ?                           


1

In [ ]:
gradonly_model = LitSirenNet(hidden_features=256, hidden_layers=3, first_omega_0=1.5, hidden_omega_0=11.7, squared_slowness=0.23,
                             steps_per_vid_chunk=100, learning_rate=1e-4, grad_loss_scale=1e-4, wave_loss_scale=0)
trainer.fit(gradonly_model)

### Now get optuna optimization trials working
The most well-regularized hyperparameters for simple (`mse_loss` only) ended up being 
256 hidden features, 3.7995 first_omega_0, 2.9312 hidden_omega_0

With wavefunc loss, `squared_slowness` of around 0.5 may be close

```
Finished trial#26 with value: 0.09781524538993835 with parameters: 
{'first_omega_0': 4.839289222946841, 'hidden_omega_0': 13.756932872278343, 'squared_slowness': 0.27488941275825124, 'wave_loss_scale': 9.252313787089657e-08}
```

In [ ]:
import os
import torch
import pytorch_lightning as pl
from surfbreak.waveform_models import LitSirenNet
from optuna.integration import PyTorchLightningPruningCallback
from surfbreak.studies import run_waveform_hyperparam_search, MetricsCallback
LOGDIR = 'logs'
MODELDIR = os.path.join(LOGDIR, 'opt_models')

def objective(trial):
    checkpoint_callback = pl.callbacks.ModelCheckpoint( # Filenames for each trial must be made unique
        os.path.join(MODELDIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss")
    tb_logger = pl.loggers.TensorBoardLogger(LOGDIR+'/', name="optuna")
    metrics_callback = MetricsCallback()     # Simple callback that saves metrics from each validation step.

    trainer = pl.Trainer(logger=tb_logger,
        limit_val_batches=1,
        checkpoint_callback=checkpoint_callback,
        max_epochs=10,
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback],
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_loss"),
        )
    # Blurry but good hyperparams: fo 1.1492  ho 4.0764  ss 0.20449  wls 4.6813e-7 
    #   Avg of top 8 hyperparams: fo 3.8960  ho 17.185  ss 0.26873  wls 5.8704e-8
    #     sharp, even better?      fo 1.5191  ho 11.694  ss 0.22755  wls 4.3596e-8
    model = LitSirenNet(hidden_features=256, # trial.suggest_categorical('hidden_features', [128, 256]),
                        hidden_layers=3,
                        first_omega_0=trial.suggest_loguniform('first_omega_0', 1, 5.),
                        hidden_omega_0=trial.suggest_loguniform('hidden_omega_0', 3, 20.), 
                        squared_slowness=trial.suggest_loguniform('squared_slowness',0.2, 0.5),
                        steps_per_vid_chunk=100, 
                        learning_rate=1e-4,
                        grad_loss_scale=0, 
                        wave_loss_scale=trial.suggest_loguniform('wave_loss_scale', 1e-8, 2e-7),
                        ) 
    trainer.fit(model)
    return metrics_callback.metrics[-1]["val_loss"].item()


study = run_waveform_hyperparam_search(objective, n_trials=100, timeout=8*60*60, model_dir=MODELDIR, prune=True, n_startup_trials=3)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]
/home/erik/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning:

The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.



[I 2020-07-02 08:51:50,247] Finished trial#0 with value: 0.12297086417675018 with parameters: {'first_omega_0': 3.6458202659161816, 'hidden_omega_0': 7.846046755759865, 'squared_slowness': 0.38767379115957384, 'wave_loss_scale': 2.0382195288893103e-08}. Best is trial#0 with value: 0.12297086417675018.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]


[I 2020-07-02 09:00:05,573] Finished trial#1 with value: 0.12134047597646713 with parameters: {'first_omega_0': 1.8474920082195172, 'hidden_omega_0': 8.407650433885147, 'squared_slowness': 0.4319669639742788, 'wave_loss_scale': 2.1546993412948287e-08}. Best is trial#1 with value: 0.12134047597646713.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]


[I 2020-07-02 09:03:56,273] Finished trial#2 with value: 0.11457269638776779 with parameters: {'first_omega_0': 1.768866259978028, 'hidden_omega_0': 8.644561375173064, 'squared_slowness': 0.4323537198032661, 'wave_loss_scale': 1.930014082589509e-08}. Best is trial#2 with value: 0.11457269638776779.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]


[I 2020-07-02 09:07:42,743] Finished trial#3 with value: 0.11480684578418732 with parameters: {'first_omega_0': 1.6500304431146329, 'hidden_omega_0': 5.537850814514927, 'squared_slowness': 0.26027949804903366, 'wave_loss_scale': 5.3897181436838255e-08}. Best is trial#2 with value: 0.11457269638776779.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]


...

[I 2020-07-02 16:27:26,342] Finished trial#67 with value: 0.10031379014253616 with parameters: {'first_omega_0': 2.3572537251052728, 'hidden_omega_0': 12.784810391465049, 'squared_slowness': 0.2851540594046142, 'wave_loss_scale': 4.602396192028105e-08}. Best is trial#21 with value: 0.09471269696950912.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]


[I 2020-07-02 16:34:00,038] Finished trial#68 with value: 0.1069357842206955 with parameters: {'first_omega_0': 2.0436328591307884, 'hidden_omega_0': 13.541595974245993, 'squared_slowness': 0.2048972543225932, 'wave_loss_scale': 2.683310413532383e-08}. Best is trial#21 with value: 0.09471269696950912.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]


[I 2020-07-02 16:39:36,410] Finished trial#69 with value: 0.10923401266336441 with parameters: {'first_omega_0': 2.660968639607584, 'hidden_omega_0': 11.761579480795753, 'squared_slowness': 0.25179486851188576, 'wave_loss_scale': 5.145708157833647e-08}. Best is trial#21 with value: 0.09471269696950912.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params | In sizes     | Out sizes                   
------------------------------------------------------------------------------
0 | model | Siren | 198 K  | [1, 1337, 3] | [[1, 1337, 1], [1, 1337, 3]]


[I 2020-07-02 16:48:52,044] Finished trial#70 with value: 0.09951135516166687 with parameters: {'first_omega_0': 1.541677607987008, 'hidden_omega_0': 14.348136908360804, 'squared_slowness': 0.24409722622804747, 'wave_loss_scale': 9.609884146075095e-08}. Best is trial#21 with value: 0.09471269696950912.


Number of finished trials: 71
Best trial was #21:
  Value: 0.09471269696950912
  Params: 
    first_omega_0: 4.369044391037427
    hidden_omega_0: 12.331887275443181
    squared_slowness: 0.267509890230128
    wave_loss_scale: 1.309917259833678e-07


In [ ]:
study.best_params

{'first_omega_0': 4.369044391037427,
 'hidden_omega_0': 12.331887275443181,
 'squared_slowness': 0.267509890230128,
 'wave_loss_scale': 1.309917259833678e-07}

In [ ]:
sdf = study.trials_dataframe()
top_trials = sdf.sort_values(by='value')[:8]
top_trials

,number,value,datetime_start,datetime_complete,duration,params_first_omega_0,params_hidden_omega_0,params_squared_slowness,params_wave_loss_scale,state
21,21,0.094713,2020-07-02 10:48:00.693940,2020-07-02 10:57:27.273261,00:09:26.579321,4.369044,12.331887,0.267510,1.309917e-07,COMPLETE
26,26,0.097815,2020-07-02 11:32:07.992255,2020-07-02 11:41:32.793535,00:09:24.801280,4.839289,13.756933,0.274889,9.252314e-08,COMPLETE
59,59,0.098240,2020-07-02 15:29:02.049362,2020-07-02 15:38:15.509467,00:09:13.460105,1.988226,11.386604,0.212841,5.441307e-08,COMPLETE
56,56,0.098308,2020-07-02 15:05:51.176908,2020-07-02 15:13:16.365645,00:07:25.188737,3.492793,11.433155,0.255806,7.837254e-08,COMPLETE
27,27,0.098461,2020-07-02 11:41:32.796178,2020-07-02 11:50:54.103157,00:09:21.306979,4.318263,13.534161,0.271601,9.206230e-08,COMPLETE
45,45,0.098500,2020-07-02 13:44:08.597750,2020-07-02 13:50:47.043539,00:06:38.445789,4.248349,18.150160,0.275486,8.269524e-08,COMPLETE
58,58,0.099266,2020-07-02 15:22:28.801758,2020-07-02 15:29:02.046584,00:06:33.244826,2.095594,11.455561,0.230230,5.305761e-08,COMPLETE
64,64,0.099319,2020-07-02 16:02:51.090734,2020-07-02 16:10:23.535365,00:07:32.444631,2.454221,9.852993,0.241796,3.892532e-08,COMPLETE


In [ ]:
print(top_trials.mean())
print(top_trials.std())

number                                       44.5
value                                   0.0980777
duration                   0 days 00:08:11.933958
params_first_omega_0                      3.47572
params_hidden_omega_0                     12.7377
params_squared_slowness                   0.25377
params_wave_loss_scale                7.78801e-08
dtype: object
number                                    17.3452
value                                  0.00145161
duration                   0 days 00:01:17.186013
params_first_omega_0                      1.14176
params_hidden_omega_0                     2.52489
params_squared_slowness                 0.0233028
params_wave_loss_scale                2.91404e-08
dtype: object
